# 네이버 댓글 데이타 수집하기 

- 1 페이지 
- n 개 페이지 
- 전체 페이지 

In [1]:
# 관련모듈 임포트 
import requests
from bs4 import BeautifulSoup
import pandas as pd

## 한페이지 댓글

In [2]:
# 1) 웹페이지 주소 요청 => 문자열 데이타 => 숩객체 

url = 'https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=167697&target=after&page=1'
res = requests.get(url)

html_str = res.text

soup = BeautifulSoup(html_str, 'html.parser')
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [5]:
# 2) 파싱 
# 영화제목, 평점, 댓글, 작성자, 날짜 
target_list = soup.select('tbody > tr')
print(len(target_list))
# print(target_list[0])

10


In [24]:
# 데이타 추출 
# 영화제목, 평점, 댓글, 작성자, 날짜 
print('제목 :', target_list[0].select_one('a').text)
print('평점 :', target_list[0].em.text)
# br 태그의 다음 요소로 접근
print('댓글 :', target_list[0].select_one('br').next_sibling.strip())
print('작성자 :', target_list[0].select_one('.author').text)
# td 태그 3번째 에서 br 태그 다음요소 접근
print('날짜 :', target_list[0].select('td')[2].select_one('br').next_sibling)

제목 : 신과함께-인과 연
평점 : 10
댓글 : 지루하지않고 재밌었다인물간 얽혀있는 이야기를 보여주는 방식이 좋았음
작성자 : oliv****
날짜 : 23.01.23


In [ ]:
<tr>
	<td class="ac num">18699331</td>
	<td class="title">
		<a href="/movie/bi/mi/basic.naver?code=167697" class="movie color_b">신과함께-인과 연</a>
			<div class="list_netizen_score">
				<span class="st_off"><span class="st_on" style="width:100%">별점 - 총 10점 중</span></span><em>10</em>
			</div>
			<br>지루하지않고 재밌었다인물간 얽혀있는 이야기를 보여주는 방식이 좋았음 
			<a href="#" onclick="report('oliv****', 'b302SgPzvFGwOd7jHwYDNYh2oahD4BHjDPjmxmV4xzk=', '지루하지않고 재밌었다인물간 얽혀있는 이야기를 보여주는 방식이 좋았음', '18699331', 'point_after');" class="report" style="color:#8F8F8F" title="새 창">신고</a>
	</td>
	<td class="num"><a href="javascript:find_list('nickname','18699331');" class="author">oliv****</a><br>23.01.23</td>
</tr>

In [26]:
# 3) 테이블 구조 데이타 ( 2차원 리스트 )
result_list = []
for tag in target_list:
    title = tag.select_one('a').text
    score = tag.em.text
    contents = tag.select_one('br').next_sibling.strip()
    writer = tag.select_one('.author').text
    date = tag.select('td')[2].select_one('br').next_sibling
    result_list.append([title, score, writer, date, contents])

In [28]:
# 4) 판다스의 데이타프레임 구조로 변경하기 
df = pd.DataFrame(result_list, 
                  columns=['영화 제목', '평점', '작성자','날짜', '댓글'])
df

,영화 제목,평점,작성자,날짜,댓글
0,신과함께-인과 연,10,oliv****,23.01.23,지루하지않고 재밌었다인물간 얽혀있는 이야기를 보여주는 방식이 좋았음
1,신과함께-인과 연,2,ckdg****,23.01.10,뇌절에 뇌절…뇌빼고만드신줄
2,신과함께-인과 연,7,greg****,23.01.08,음. 좋다 마른 영화 좀 별로...
3,신과함께-인과 연,10,kkin****,23.01.02,대박...이란말밖에
4,신과함께-인과 연,3,elec****,23.01.01,아무리 봐도 내 스타일의 영화는 아니다.
5,신과함께-인과 연,8,epah****,22.12.28,판 벌여놓고 그대로 튈생각은 아니겠지
6,신과함께-인과 연,10,hune****,22.12.25,반전도 조금 있고 한번쯤은 꼭봐야하는 영화
7,신과함께-인과 연,10,hune****,22.12.25,반전도 조금 있고 한번쯤은 꼭봐야하는 영화
8,신과함께-인과 연,10,hune****,22.12.25,반전도 조금 있고 한번쯤은 꼭봐야하는 영화
9,신과함께-인과 연,6,harr****,22.12.04,투섬스업!!투썸서업!


## 여러 페이지 댓글 - 범위가 정해진 경우 

### 페이지 주소 만들기 

In [31]:
base_url = 'https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=167697&target=after&page='
# n개의 페이지 주소 
n = 5
# 결과 리스트 초기화 
result_list = []

for i in range(1, n+1):
    # 1) 주소 생성 
    movie_url = base_url+str(i)
    print(movie_url)
    
    # 2) 데이타요청 => 문자열 => 숩객체
    res = requests.get(movie_url)
    html_str = res.text
    soup = BeautifulSoup(html_str, 'html.parser')
    
    # 3) 파싱
    target_list = soup.select('tbody > tr')
    
    # 4) 테이블 구조 데이타 ( 2차원 리스트 )
    for tag in target_list:
        title = tag.select_one('a').text
        score = tag.em.text
        contents = tag.select_one('br').next_sibling.strip()
        writer = tag.select_one('.author').text
        date = tag.select('td')[2].select_one('br').next_sibling
        
        result_list.append([title, score, writer, date, contents])

https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=167697&target=after&page=1
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=167697&target=after&page=2
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=167697&target=after&page=3
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=167697&target=after&page=4
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=167697&target=after&page=5


In [32]:
# 댓글 갯수 확인 
print(len(result_list))

50


In [34]:
# 5) 판다스의 데이타프레임 구조로 변경하기 
df = pd.DataFrame(result_list, 
                  columns=['영화 제목', '평점', '작성자','날짜', '댓글'])
df.tail(3)

,영화 제목,평점,작성자,날짜,댓글
47,신과함께-인과 연,10,bbbh****,22.02.12,
48,신과함께-인과 연,4,head****,22.01.31,신과 함께는신파 랑께
49,신과함께-인과 연,7,acec****,22.01.31,억지 감성팔이가 줄어서 좋았습니다


In [35]:
# 6) csv 파일로 저장 
df.to_csv('output/신과함께_댓글.csv', index=False, encoding='cp949')

## 전체 페이지 댓글 

### 다음 버튼 파싱

- url : 키드 
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=1

In [39]:
# 1) 웹페이지 주소 요청 => 문자열 데이타 => 숩객체 
url = 'https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=1'
res = requests.get(url)
html_str = res.text
soup = BeautifulSoup(html_str, 'html.parser')

# 2) 다음 버튼 파싱 
next_btn = soup.select_one('.pg_next')
print(next_btn)
# 없다면 (마지막 페이지) none

<a class="pg_next" href="/movie/point/af/list.naver?st=mcode&amp;sword=13558&amp;target=after&amp;page=2" title="다음"><em>다음</em></a>


In [ ]:
# 다음 페이지 버튼 소스 확인 
<a href="/movie/point/af/list.naver?st=mcode&amp;sword=13558&amp;target=after&amp;page=2" 
title="다음" class="pg_next">
    <em>다음</em>
</a>

### 페이지 주소 만들기 

In [47]:
# 페이지 주소 확인 
# 키드 = 13558
base_url = 'https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page='
page = 1
# 결과 리스트 초기화 
result_list = []

while True:
    # 1) 주소 생성 
    movie_url = base_url+str(page)
    print(movie_url)
    
    # 2) 페이지 요청 후 숩 객체 생성 
    res = requests.get(movie_url)
    html_str = res.text
    soup = BeautifulSoup(html_str, 'html.parser')
    
    # 3) 다음 버튼 파싱 
    next_btn = soup.select_one('.pg_next')  
    
    # 4) 마지막 페이지인지 검사
    if next_btn == None:
        break
        
    # 페이지 번호 증가     
    page += 1

https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=1
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=2
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=3
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=4
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=5
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=6
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=7
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=8
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=9
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=10
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=

### 전체 페이지 데이타 저장

In [48]:
# 페이지 주소 확인 
# 키드 = 13558
base_url = 'https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page='
page = 1
# 결과 리스트 초기화 
result_list = []

while True:
    # 1) 주소 생성 
    movie_url = base_url+str(page)
    print(movie_url)
    
    # 2) 페이지 요청 후 숩 객체 생성 
    res = requests.get(movie_url)
    html_str = res.text
    soup = BeautifulSoup(html_str, 'html.parser')
    
    # 3)  파싱 
    next_btn = soup.select_one('.pg_next')  
    target_list = soup.select('tbody > tr')
    
    # 4) 테이블 구조 데이타 ( 2차원 리스트 )
    for tag in target_list:
        title = tag.select_one('a').text
        score = tag.em.text
        contents = tag.select_one('br').next_sibling.strip()
        writer = tag.select_one('.author').text
        date = tag.select('td')[2].select_one('br').next_sibling
        
        result_list.append([title, score, writer, date, contents])
     
    # 5) 마지막 페이지인지 검사
    if next_btn == None:
        break
        
    # 6) 페이지 번호 증가     
    page += 1

https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=1
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=2
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=3
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=4
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=5
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=6
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=7
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=8
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=9
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=10
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=

In [49]:
# 전체 갯수 확인 
len(result_list)

146

In [50]:
# 7) 판다스의 데이타프레임 구조로 변경하기 
df = pd.DataFrame(result_list, 
                  columns=['영화 제목', '평점', '작성자','날짜', '댓글'])
df.tail(3)

,영화 제목,평점,작성자,날짜,댓글
143,키드,10,pby1****,04.10.10,감동적인 드라마.
144,키드,10,litt****,04.09.16,웃음으로 포장한 가슴 아픈 비극
145,키드,9,sjzs****,03.12.26,따뜻한 아빠 채플린과 귀여운 아이


In [51]:
# 8) csv 파일로 저장 
df.to_csv('output/키드_댓글.csv', index=False, encoding='cp949')

# 퀴즈 - 슬라이드 116

- Yes24 국내도서 종합 베스트 셀러
- http://www.yes24.com/24/category/bestseller?CategoryNumber=001&sumgb=06&fetchSize=40&PageNumber=1
- 1~999위 까지 CSV 파일로 저장하여라
- 순위 / 책제목 / 저자 / 정가 / 할인가 / 책 상세 URL / 표지 URL

### 주소 만들기 

In [65]:
# 페이지 주소 확인 
base_url = 'http://www.yes24.com/24/category/bestseller?categorynumber=001&sumgb=06&fetchsize=40&pagenumber='
page = 0
# 결과 리스트 초기화 
result_list = []

while True:
    # 1) 주소 생성
    # 페이지 번호 증가     
    page += 1
    url = base_url+str(page)
    
    # 2) 페이지 요청 후 숩 객체 생성 
    res = requests.get(url)
    html_str = res.text
    soup = BeautifulSoup(html_str, 'html.parser')
    
    # 3) 파싱 
    contents = soup.select('tr')  
    
    # 4) 마지막 페이지인지 검사
    if len(contents):
        print(url)
    else:
        break

http://www.yes24.com/24/category/bestseller?categorynumber=001&sumgb=06&fetchsize=40&pagenumber=1
http://www.yes24.com/24/category/bestseller?categorynumber=001&sumgb=06&fetchsize=40&pagenumber=2
http://www.yes24.com/24/category/bestseller?categorynumber=001&sumgb=06&fetchsize=40&pagenumber=3
http://www.yes24.com/24/category/bestseller?categorynumber=001&sumgb=06&fetchsize=40&pagenumber=4
http://www.yes24.com/24/category/bestseller?categorynumber=001&sumgb=06&fetchsize=40&pagenumber=5
http://www.yes24.com/24/category/bestseller?categorynumber=001&sumgb=06&fetchsize=40&pagenumber=6
http://www.yes24.com/24/category/bestseller?categorynumber=001&sumgb=06&fetchsize=40&pagenumber=7
http://www.yes24.com/24/category/bestseller?categorynumber=001&sumgb=06&fetchsize=40&pagenumber=8
http://www.yes24.com/24/category/bestseller?categorynumber=001&sumgb=06&fetchsize=40&pagenumber=9
http://www.yes24.com/24/category/bestseller?categorynumber=001&sumgb=06&fetchsize=40&pagenumber=10
http://www.yes24.co

In [61]:
# 페이지 주소 확인 
base_url = 'http://www.yes24.com/24/category/bestseller?categorynumber=001&sumgb=06&fetchsize=40&pagenumber='
page = 26
url = base_url+str(page)
print(url)

res = requests.get(url)
html_str = res.text
soup = BeautifulSoup(html_str, 'html.parser')
contents = soup.select('tr')
print(contents)

http://www.yes24.com/24/category/bestseller?categorynumber=001&sumgb=06&fetchsize=40&pagenumber=26
[]
